In [3]:
!pip install plotly kaleido datasets -U -q


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import os

import datasets
import pandas as pd
from dotenv import load_dotenv
from huggingface_hub import login


load_dotenv(override=True)
login(os.getenv("HF_TOKEN"))

pd.set_option("max_colwidth", None)

OUTPUT_DIR = "output"

/Users/aymeric/venv/test/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
eval_ds = datasets.load_dataset("gaia-benchmark/GAIA", "2023_all")["validation"]
eval_ds = eval_ds.rename_columns({"Question": "question", "Final answer": "true_answer", "Level": "task"})
eval_df = pd.DataFrame(eval_ds)

In [6]:
pd.Series(eval_ds["task"]).value_counts()

2    86
1    53
3    26
Name: count, dtype: int64

# 1. Load all results

In [7]:
import glob


answer_file_path = f"{OUTPUT_DIR}/validation/answers.jsonl"

result_df = pd.concat(
    [pd.read_json(f, lines=True) for f in glob.glob(f"{OUTPUT_DIR}/validation/*.jsonl") if "answers.jsonl" not in f]
)
result_df = result_df.drop(columns=["start_time", "end_time"])
result_df.to_json(answer_file_path, lines=True, orient="records")

In [8]:
import re
from collections import Counter

from scripts.gaia_scorer import check_close_call, question_scorer


result_df["is_correct"] = result_df.apply(lambda x: question_scorer(x["prediction"], x["true_answer"]), axis=1)
result_df["is_near_correct"] = result_df.apply(
    lambda x: check_close_call(x["prediction"], x["true_answer"], x["is_correct"]),
    axis=1,
)

result_df["count_steps"] = result_df["intermediate_steps"].apply(len)


def find_attachment(question):
    matches = eval_df.loc[eval_df["question"].apply(lambda x: x in question), "file_name"]

    if len(matches) == 0:
        return "Not found"
    file_path = matches.values[0]

    if isinstance(file_path, str) and len(file_path) > 0:
        return file_path.split(".")[-1]
    else:
        return "None"


result_df["attachment_type"] = result_df["question"].apply(find_attachment)


def extract_tool_calls(code):
    regex = r"\b(\w+)\("
    function_calls = [el for el in re.findall(regex, code) if el.islower()]

    function_call_counter = Counter(function_calls)
    return function_call_counter


def sum_tool_calls(steps):
    total_count = Counter()
    for step in steps:
        if "llm_output" in step:
            total_count += extract_tool_calls(step["llm_output"])

    return total_count


# result_df["tool_calls"] = result_df["intermediate_steps"].apply(sum_tool_calls)

String 250 for Cheater cannot be normalized to number str.
String  220 for Cheater beater cannot be normalized to number str.
Close call: INT. THE CASTLE vs THE CASTLE
Close call: to be or not to be that is the question whether tis nobler in the mind to suffer the slings and arrows of outrageous fortune or to take arms against a sea of troubles and by opposing end them vs To be or not to be that is the question whether tis nobler in the mind to suffer the slings and arrows of outrageous fortune


/Users/aymeric/Documents/Code/smolagents/examples/GAIA_submission/scripts/gaia_scorer.py:52: UserWarning: Answer lists have different lengths, returning False.
  warnings.warn("Answer lists have different lengths, returning False.", UserWarning)


In [9]:
def get_thoughts(x):
    try:
        output = x[0]["task"]
        for y in x[1:]:
            try:
                if "observation" in y:
                    output += y["llm_output"] + "\nObservation:" + y["observation"]
                else:
                    output += y["llm_output"] + r"\Error:" + str(y["error"])
            except:
                pass
        return output
    except:
        return None


result_df["thoughts"] = result_df["intermediate_steps"].apply(lambda x: get_thoughts(x))

In [10]:
result_df["agent_name"].value_counts()

agent_name
code_o1_22-01_managedagent-summary_planning    67
code_o1_25-01_visioon                          53
Name: count, dtype: int64

# 2. Inspect specific runs

In [15]:
o1 = "code_o1_22-01_managedagent-summary_planning"
o1_vision = "code_o1_25-01_visioon"

list_versions = [o1, o1_vision]

# submission_selection_name = "react_code_llama3-70b_02-05_full-gaia-validation-code"
sel_df = result_df.loc[
    (result_df["agent_name"].isin(list_versions))
    # & (~result_df["question"].isin(UNSOLVED_QUESTIONS))
].reset_index(drop=True)
display(sel_df["agent_name"].value_counts())
sel_df = sel_df.drop_duplicates(subset=["agent_name", "question"])
display(sel_df.groupby("agent_name")[["task"]].value_counts())
print("Total length:", len(sel_df), "- is complete:", len(sel_df) == 165)
# assert sel_df["question"].value_counts().max() == len(list_versions), "Some questions are duplicate!"

agent_name
code_o1_22-01_managedagent-summary_planning    67
code_o1_25-01_visioon                          53
Name: count, dtype: int64

agent_name                                   task
code_o1_22-01_managedagent-summary_planning  2       36
                                             1       21
                                             3       10
code_o1_25-01_visioon                        2       30
                                             1       17
                                             3        6
Name: count, dtype: int64

Total length: 120 - is complete: False


In [16]:
sel_df.loc[
    (sel_df["is_correct"] == False) & (sel_df["is_near_correct"] == True),
    ["question", "prediction", "true_answer"],
]

,question,prediction,true_answer
21,"In Series 9, Episode 11 of Doctor Who, the Doctor is trapped inside an ever-shifting maze. What is this location called in the official script for the episode? Give the setting exactly as it appears in the first scene heading.",INT. THE CASTLE,THE CASTLE
44,"Could you help me out with this assignment? Our professor sprung it on us at the end of class Friday, and I'm still trying to figure it out. The question he asked us was about an anagram. I've attached an audio recording of the question that he asked, so if you could please take a listen and give me the answer, I'd really appreciate the help. Please limit your response to the anagram text that could be generated from the original line which fulfills the professor's request, without any other commentary. Also, please don't include any punctuation in your response.",to be or not to be that is the question whether tis nobler in the mind to suffer the slings and arrows of outrageous fortune or to take arms against a sea of troubles and by opposing end them,To be or not to be that is the question whether tis nobler in the mind to suffer the slings and arrows of outrageous fortune


In [17]:
display("Average score:", sel_df.groupby("agent_name")[["is_correct"]].mean().round(3))
display(
    sel_df.groupby(["agent_name", "task"])[["is_correct", "is_near_correct", "count_steps", "question"]]
    .agg(
        {
            "is_correct": "mean",
            "is_near_correct": "mean",
            "count_steps": "mean",
            "question": "count",
        }
    )
    .rename(columns={"question": "count"})
)

'Average score:'

,is_correct
agent_name,
code_o1_22-01_managedagent-summary_planning,0.418
code_o1_25-01_visioon,0.340


is_correct  is_near_correct  \
agent_name                                  task                                
code_o1_22-01_managedagent-summary_planning 1       0.476190         0.523810   
                                            2       0.472222         0.500000   
                                            3       0.100000         0.100000   
code_o1_25-01_visioon                       1       0.411765         0.411765   
                                            2       0.366667         0.366667   
                                            3       0.000000         0.000000   

                                                  count_steps  count  
agent_name                                  task                      
code_o1_22-01_managedagent-summary_planning 1        5.047619     21  
                                            2        5.222222     36  
                                            3        5.500000     10  
code_o1_25-01_visioon                       1        5.294118     17  
                                            2        5.333333     30  
                                            3        6.666667      6

In [18]:
import plotly.express as px


cumulative_df = (
    (
        sel_df.groupby("agent_name")[["is_correct", "is_near_correct"]]
        .expanding(min_periods=1, axis=0, method="single")
        .agg({"is_correct": "mean", "is_near_correct": "count"})
        .reset_index()
    )
    .copy()
    .rename(columns={"is_near_correct": "index"})
)
cumulative_df["index"] = cumulative_df["index"].astype(int) - 1


def find_question(row):
    try:
        res = sel_df.loc[sel_df["agent_name"] == row["agent_name"], "question"].iloc[row["index"]][:50]
        return res
    except Exception:
        return ""


cumulative_df["question"] = cumulative_df.apply(find_question, axis=1)
# cumulative_df["question"] = [el[:50] for el in sel_df["question"].values]

# cumulative_df["is_correct"] = cumulative_df["is_correct"] * (165 - 68) / 165

px.line(
    cumulative_df,
    color="agent_name",
    x="index",
    y="is_correct",
    hover_data="question",
)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([['A paper about AI regulation that was originally su'],
                                   ['I’m researching species that became invasive after'],
                                   ['If we assume all articles published by Nature in 2'],
                                   ['In Unlambda, what exact charcter or text needs to '],
                                   ['If Eliud Kipchoge could maintain his record-making'],
                                   ['How many studio albums were published by Mercedes '],
                                   ["The object in the British Museum's collection with"],
                                   ['According to github, when was Regression added to '],
                                   ["Here's a fun riddle that I think you'll enjoy.\n\nYo"],
                                   ['In July 2, 1959 United States standards for grades'],
                                   ['Using the Biopython library in Python, parse the P'],
                                   ['What are the EC numbers of the two most commonly u'],
                                   ['In April of 1977, who was the Prime Minister of th'],
                                   ["What's the last line of the rhyme under the flavor"],
                                   ['Use density measures from the chemistry materials '],
                                   ['What was the volume in m^3 of the fish bag that wa'],
                                   ['What is the average number of pre-2020 works on th'],
                                   ['In the video https://www.youtube.com/watch?v=L1vXC'],
                                   ['Of the authors (First M. Last) that worked on the '],
                                   ['When you take the average of the standard populati'],
                                   ['Assuming scientists in the famous youtube video Th'],
                                   ['In Series 9, Episode 11 of Doctor Who, the Doctor '],
                                   ['In terms of geographical distance between capital '],
                                   ['In the NCATS PubChem compound database for Food Ad'],
                                   ['I need to fact-check a citation. This is the citat'],
                                   ['Which contributor to the version of OpenCV where s'],
                                   ['What integer-rounded percentage of the total lengt'],
                                   ['An office held a Secret Santa gift exchange where '],
                                   ['What is the maximum length in meters of #9 in the '],
                                   ['What two-word type of model did Manash Pratim Kash'],
                                   ['What animals that were mentioned in both Ilias Lag'],
                                   ['How many High Energy Physics - Lattice articles li'],
                                   ['The photograph in the Whitney Museum of American A'],
                                   ['.rewsna eht sa "tfel" drow eht fo etisoppo eht eti'],
                                   ['What is the minimum number of page links a person '],
                                   ['I went to Virtue restaurant & bar in Chicago for m'],
                                   ['¬(A ∧ B) ↔ (¬A ∨ ¬B)\n¬(A ∨ B) ↔ (¬A ∧ ¬B)\n(A → B) '],
                                   ['My family reunion is this week, and I was assigned'],
                                   ["In Emily Midkiff's June 2014 article in a journal "],
                                   ['It is 1999. Before you party like it is 1999, plea'],
                                   ["Under DDC 633 on Bielefeld University Library's BA"],
                                   ['In the 2018 VSCode blog post on replit.com, what w'],
                                   ['Compute the check digit the Tropicos ID for the Or'],
                                   ['What time was the Tri-Rail train that

# 3. Dive deeper into one run

In [18]:
sel_df = result_df.loc[result_df["agent_name"] == o1]
print(len(sel_df))

49


### Count errors

In [19]:
import numpy as np


error_types = [
    "AgentParsingError",
    "AgentExecutionError",
    "AgentMaxIterationsError",
    "AgentGenerationError",
]
sel_df[error_types] = 0
sel_df["Count steps"] = np.nan


def count_errors(row):
    if isinstance(row["intermediate_steps"], list):
        row["Count steps"] = len(row["intermediate_steps"])
        for step in row["intermediate_steps"]:
            if isinstance(step, dict) and "error" in step:
                try:
                    row[str(step["error"]["error_type"])] += 1
                except:
                    pass
    return row


sel_df = sel_df.apply(count_errors, axis=1)

In [22]:
import plotly.express as px


aggregate_errors = (
    sel_df.groupby(["is_correct"])[error_types + ["Count steps"]].mean().reset_index().melt(id_vars=["is_correct"])
)

fig = px.bar(
    aggregate_errors,
    y="value",
    x="variable",
    color="is_correct",
    labels={
        "agent_name": "<b>LLM Engine</b>",
        "task": "<b>Level</b>",
        "aggregate_score": "<b>Performance</b>",
        "value": "<b>Average count</b>",
        "eval_score_GPT4": "<b>Score</b>",
    },
)
fig.update_layout(
    height=500,
    width=800,
    barmode="group",
    bargroupgap=0.0,
)
fig.update_traces(textposition="outside")
fig.write_image("figures/aggregate_errors.png", scale=3)
fig.show()

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


### Count tool calls

In [15]:
tools_calls = pd.DataFrame.from_records(sel_df["tool_calls"].values).fillna(0)

# Exclude the tools that were not used enough
tools_calls = tools_calls.loc[:, tools_calls.sum() > 10]

# Sort the columns by the sum of the values
tools_calls = tools_calls[tools_calls.sum().sort_values(ascending=False).index]
display(tools_calls)
sel_with_calls = pd.concat([sel_df[["question", "is_correct", "task"]], tools_calls], axis=1)
sel_with_calls = sel_with_calls.drop("question", axis=1).groupby(["is_correct", "task"]).mean()
# sel_with_calls = sel_with_calls.melt(id_vars=['question', 'is_correct', 'task'], var_name="tool", value_name='count')

,print,ask_search_agent,final_answer,len,range,inspect_file_as_text,set,visualizer,parse_square,sum,...,max,join,generate_prefixes,sorted,get,lower,f,search_birthdate,items,abs
0,8.0,3.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,5.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161,3.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162,7.0,2.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
163,20.0,8.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
sel_with_calls = sel_with_calls.reset_index().melt(
    id_vars=["is_correct", "task"], var_name="tool", value_name="average_count"
)

In [17]:
import plotly.express as px


fig = px.bar(
    sel_with_calls,
    x="tool",
    y="average_count",
    color="is_correct",
    facet_row="task",
    labels={
        "agent_name": "<b>Agent variant</b>",
        "task": "<b>Level</b>",
        "aggregate_score": "<b>Performance</b>",
        "eval_score_GPT4": "<b>Score</b>",
        "agent_type": "<b>Agent type</b>",
        "average_count": "<b>Average #calls per run</b>",
    },
)
fig.update_layout(
    barmode="group",
    height=800,
    width=1000,
    title="<b>" + "</b>",
)

### Inspect result by file extension type

In [18]:
display(
    sel_df.groupby(["agent_name", "attachment_type"])[["is_correct", "count_steps", "question"]].agg(
        {"is_correct": "mean", "count_steps": "mean", "question": "count"}
    )
)

is_correct  \
agent_name                                    attachment_type               
react_code_gpt4o_23-june_planning2_newprompt5 None               0.440945   
                                              csv                0.000000   
                                              docx               0.000000   
                                              jpg                0.000000   
                                              jsonld             0.000000   
                                              mp3                1.000000   
                                              pdb                0.000000   
                                              pdf                0.333333   
                                              png                0.125000   
                                              pptx               1.000000   
                                              py                 1.000000   
                                              txt                0.000000   
                                              xlsx               0.615385   
                                              zip                1.000000   

                                                               count_steps  \
agent_name                                    attachment_type                
react_code_gpt4o_23-june_planning2_newprompt5 None                9.196850   
                                              csv                 7.000000   
                                              docx                9.000000   
                                              jpg                 9.500000   
                                              jsonld             16.000000   
                                              mp3                 8.333333   
                                              pdb                 7.000000   
                                              pdf                 5.666667   
                                              png                 6.750000   
                                              pptx                4.000000   
                                              py                  4.000000   
                                              txt                 6.000000   
                                              xlsx                7.538462   
                                              zip                10.000000   

                                                               question  
agent_name                                    attachment_type            
react_code_gpt4o_23-june_planning2_newprompt5 None                  127  
                                              csv                     1  
                                              docx                    1  
                                              jpg                     2  
                                              jsonld                  1  
                                              mp3                     3  
                                              pdb                     1  
                                              pdf                     3  
                                              png                     8  
                                              pptx                    1  
                                              py                      1  
                                              txt                     1  
                                              xlsx                   13  
                                              zip                     2

In [44]:
# Inspect specific file types
# sel_df.loc[
#     sel_df["attachment_type"].isin(["pdb", "docx", "csv"]),
#     [
#         "attachment_type",
#         "question",
#         "prediction",
#         "true_answer",
#         "is_correct",
#         "thoughts",
#     ],
# ]

# 4. Ensembling methods

### 4.1 Simple retry mechanism

In [54]:
first_run_gpt4 = result_df.loc[result_df["agent_name"] == gpt4o].copy()
second_run_gpt4 = result_df.loc[result_df["agent_name"] == noanchorplan].copy()


def replace_answer_if_incomplete(row, result_df_replacement):
    try:
        if (
            "Unable to determine" in row["intermediate_steps"]
            or "AgentMaxIterationsError" in str(row["intermediate_steps"])
            # or "AgentExecutionError" in str(row["intermediate_steps"])
            # or "AgentGenerationError" in str(row["intermediate_steps"])
            or "Error in generating final llm output" in str(row["intermediate_steps"])
        ):
            matching_answer = result_df_replacement.loc[
                (result_df_replacement["question"] == row["question"]), "prediction"
            ].values[0]
            print("replaced")
            gold_answer = matching_answer
        else:
            gold_answer = row["prediction"]
    except:
        gold_answer = row["prediction"]
    return gold_answer


combined_gpt4 = first_run_gpt4.copy()
combined_gpt4["prediction"] = combined_gpt4.apply(lambda x: replace_answer_if_incomplete(x, second_run_gpt4), axis=1)

combined_gpt4["is_correct"] = combined_gpt4.apply(lambda x: question_scorer(x["prediction"], x["true_answer"]), axis=1)

replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced


In [55]:
print("First run:")
print(first_run_gpt4.groupby(["task"])["is_correct"].mean())
print(first_run_gpt4["is_correct"].mean())

print("Second run:")
print(second_run_gpt4.groupby(["task"])["is_correct"].mean())
print(second_run_gpt4["is_correct"].mean())

print("Combined run:")
print(combined_gpt4.groupby(["task"])["is_correct"].mean())
print(combined_gpt4["is_correct"].mean())

First run:
task
1    0.566038
2    0.418605
3    0.200000
Name: is_correct, dtype: float64
0.4329268292682927
Second run:
task
1    0.528302
2    0.372093
3    0.200000
Name: is_correct, dtype: float64
0.39634146341463417
Combined run:
task
1    0.566038
2    0.395349
3    0.160000
Name: is_correct, dtype: float64
0.4146341463414634


### 4.2 Ideal ensembling

In [87]:
third_run = result_df.loc[result_df["agent_name"] == noanchorplan].copy()
INCLUDE_THIRD_RUN = False


# test ideal ensembling
def score_best_both(row, result_df_replacement):
    try:
        if row["is_correct"]:
            return True

        else:
            matching_answer = result_df_replacement.loc[(result_df_replacement["question"] == row["question"])].iloc[0]
            if matching_answer["is_correct"]:
                return True
            else:
                return False
    except:
        return row["is_correct"]


combined_gpt4 = first_run_gpt4.copy()
combined_gpt4["is_correct"] = combined_gpt4.apply(lambda x: score_best_both(x, second_run_gpt4), axis=1)
if INCLUDE_THIRD_RUN:
    combined_gpt4["is_correct"] = combined_gpt4.apply(lambda x: score_best_both(x, third_run), axis=1)
print("Ideal combined run:")
print(combined_gpt4.groupby(["task"])["is_correct"].mean())
print(combined_gpt4["is_correct"].mean())

Ideal combined run:
task
1    0.641509
2    0.465116
3    0.240000
Name: is_correct, dtype: float64
0.4878048780487805
